In [1]:
import vrep # access all the VREPelements
import sys
import time
import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
#import scipy
from math import sin, cos, pi
import random as r
import math

ModuleNotFoundError: No module named 'vrep'

In [ ]:
# Put Constants here (e.g. sensors name, limit of time)

ROBOT = "Pioneer_p3dx"

# Sensors
LEFT_MOTOR = 'Pioneer_p3dx_leftMotor'
RIGHT_MOTOR = 'Pioneer_p3dx_rightMotor'
SONAR_SENSORS = "Pioneer_p3dx_ultrasonicSensor"

# Wheels
LEFT_WHEEL = 'Pioneer_p3dx_leftWheel'
RIGHT_WHEEL = 'Pioneer_p3dx_rightWheel'

# Dimensions
WHEEL_DIAMETER = 195 * 10e-4 # 195mm
WHEEL_RADIUS = WHEEL_DIAMETER / 2

# Maximum distance for sonar sensors to detect
NO_DETECTION_DIST = 1

# robot stops when reach a timeout
ROBOT_TIMEOUT = 100

# Time interval in which the position of the robot will be checked/calculated.
INTERVAL_CHECK_POSITION = 0.3

# Plot limits
X_LEFT_LIM = 4.5530
X_RIGHT_LIM = -7.8010
Y_BOTTOM_LIM = 6.9940
Y_UPPER_LIM = -5.0660

# Velocity paramenter of the braitenberg function
ROBOT_VELOCITY = 2
RAIO_ROBO = 0.455/2
ang_ultrassom = [90, 50, 30, 10, -10, -30, -50, -90]
N_SENSORS = 8

In [ ]:
vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart("127.0.0.1",19997,True,True,5000,5) # start a connection
if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")

In [ ]:
# Initialize all handles here

# Robots
err_code,robot_handle = vrep.simxGetObjectHandle(clientID, ROBOT, vrep.simx_opmode_blocking)

# Motors
err_code,l_motor_handle = vrep.simxGetObjectHandle(clientID, LEFT_MOTOR, vrep.simx_opmode_blocking)
err_code,r_motor_handle = vrep.simxGetObjectHandle(clientID, RIGHT_MOTOR, vrep.simx_opmode_blocking)

# Wheels
err_code,l_wheel_handle = vrep.simxGetObjectHandle(clientID, LEFT_WHEEL, vrep.simx_opmode_blocking)
err_code,r_wheel_handle = vrep.simxGetObjectHandle(clientID, RIGHT_WHEEL, vrep.simx_opmode_blocking)

# Sensors
sonar_sensor_handles = []
for i in range(1, N_SENSORS+1):
    err_code,sonar_handle = vrep.simxGetObjectHandle(clientID, SONAR_SENSORS + str(i), vrep.simx_opmode_blocking)
    sonar_sensor_handles.append(sonar_handle)

In [ ]:
# Distance between the wheels
err_code, lw = vrep.simxGetObjectPosition(clientID, l_wheel_handle, -1, vrep.simx_opmode_streaming)
while err_code != vrep.simx_return_ok:
    err_code, lw = vrep.simxGetObjectPosition(clientID, l_wheel_handle, -1, vrep.simx_opmode_buffer)
err_code, rw = vrep.simxGetObjectPosition(clientID, r_wheel_handle, -1, vrep.simx_opmode_streaming)
while err_code != vrep.simx_return_ok:
    err_code, rw = vrep.simxGetObjectPosition(clientID, r_wheel_handle, -1, vrep.simx_opmode_buffer)

DISTANCE_BETWEEN_WHEELS = distance.euclidean(lw[:2], rw[:2])

# Initial Position of the Robot
err_code, robot_pos = vrep.simxGetObjectPosition(clientID, robot_handle, -1, vrep.simx_opmode_streaming)
while err_code != vrep.simx_return_ok:
    err_code, robot_pos = vrep.simxGetObjectPosition(clientID, robot_handle, -1, vrep.simx_opmode_buffer)
    
# Only X and Y
ROBOT_INIT_XY = robot_pos[:2]
    
# Initial Orientation of the Robot
err_code, robot_ori = vrep.simxGetObjectOrientation(clientID, robot_handle, -1, vrep.simx_opmode_streaming)
while err_code != vrep.simx_return_ok:
    err_code, robot_ori = vrep.simxGetObjectOrientation(clientID, robot_handle, -1, vrep.simx_opmode_buffer)
    
# Only Gamma
ROBOT_INIT_THETA = robot_ori[2]

# get the relative positions of the sensors from the robot
sonar_positions_xy = []
for i in range(N_SENSORS):
    res, position = vrep.simxGetObjectPosition(clientID, sonar_sensor_handles[i], robot_handle, vrep.simx_opmode_streaming)
    while res != vrep.simx_return_ok :
        res, position = vrep.simxGetObjectPosition(clientID, sonar_sensor_handles[i], robot_handle, vrep.simx_opmode_buffer)

    sonar_positions_xy.append(position[:2])
    
#get the norm and the angle of the sensors relative to the robot
sonar_positions_norm_theta = []
i = 0
for [x,y] in sonar_positions_xy:
    i += 1
    norm = np.linalg.norm([x,y])
    
    if x >= 0 and y >= 0:
        theta = np.arctan(y/x)
    elif x < 0 and y >= 0:
        theta = np.pi - np.arcsin(y/norm)
    elif x < 0 and y < 0:
        theta = np.pi + np.arctan(y/x)
    elif x >= 0 and y < 0:
        theta = 2*np.pi - np.arccos(y/norm)
        
    #print(i, x, y, norm, theta, theta*180/np.pi)
        
    sonar_positions_norm_theta.append([norm, theta])

sonar_positions_orientations = np.vstack([np.asarray(sonar_positions_xy)[:,0],
                                          np.asarray(sonar_positions_xy)[:,1],
                                          np.asarray(sonar_positions_norm_theta)[:,1]]).T

# Sonar sensor transformation arrays
TS = []
for x,y,theta in sonar_positions_orientations:
    TS.append(np.array([[np.cos(theta), -np.sin(theta), x],
                        [np.sin(theta),  np.cos(theta), y],
                        [0,              0,             1]]))

In [ ]:
position_ground_truth = []
position_odometry = []
points_cloud_ground_truth = []
points_cloud_odometry = []

pontos_x = []
pontos_y = []
pontos = []
trajetoria_x = []
trajetoria_y = []
odometria_x = []
odometria_y = []

def salva_dados(dist, x_robo, y_robo, x_odom, y_odom, ang_robo):
    for i in range(len(dist)):
        if(dist[i] < NO_DETECTION_DIST):
            x = x_robo + (dist[i] + RAIO_ROBO) * math.cos((ang_ultrassom[i]*math.pi/180) + ang_robo)
            y = y_robo + (dist[i] + RAIO_ROBO) * math.sin((ang_ultrassom[i]*math.pi/180) + ang_robo)
            if [x, y] not in pontos: 
                pontos_x.append(x)
                pontos_y.append(y)
    trajetoria_x.append(x_robo)
    trajetoria_y.append(y_robo)
    odometria_x.append(x_odom)
    odometria_y.append(y_odom)

def plotar_mapa():
    plt.scatter(pontos_x, pontos_y, s=0.5)
    line1, = plt.plot(odometria_x, odometria_y, 'r--', label='Odometria')
    line2, = plt.plot(trajetoria_x, trajetoria_y, 'g', label='Ground-truth')
    plt.legend(handles=[line1, line2])
    plt.show()

In [ ]:
def set_velocity_one_motor(motor, v):
    """ set velocity to an specific motor """
    
    try:
        motor = motor.lower()

        if motor == "l" or motor == "left":
            err_code = vrep.simxSetJointTargetVelocity(clientID,l_motor_handle,v,
                                                       vrep.simx_opmode_streaming)
        elif motor == "r" or motor == "right":
            err_code = vrep.simxSetJointTargetVelocity(clientID,r_motor_handle,v,
                                                      vrep.simx_opmode_streaming)
    except Exception as e:
        raise e

def set_velocity_both_motors(v):
    """ set the same velocity to both motors """
    
    try:
        set_velocity_one_motor("l", v)
        set_velocity_one_motor("r", v)
    except Exception as e:
        raise e


def get_linear_motor_velocity():
    """ get the linear velocity from both motors"""

    try:
        # left motor
        err_code, linear_velocity_l, _ = vrep.simxGetObjectVelocity(clientID, l_motor_handle, 
                                                                    vrep.simx_opmode_streaming)
        while err_code != vrep.simx_return_ok:
            err_code, linear_velocity_l, _ = vrep.simxGetObjectVelocity(clientID, l_motor_handle, 
                                                                    vrep.simx_opmode_buffer)

        # right motor
        err_code, linear_velocity_r, _ = vrep.simxGetObjectVelocity(clientID, r_motor_handle,
                                                                    vrep.simx_opmode_streaming)
        while err_code != vrep.simx_return_ok:
            err_code, linear_velocity_l, _ = vrep.simxGetObjectVelocity(clientID, r_motor_handle, 
                                                                    vrep.simx_opmode_buffer)

        # I don't know why, but the multiplication to 10 is necessary
        vlinear_norm_l = np.linalg.norm(linear_velocity_l[:2]) * 10 
        vlinear_norm_r = np.linalg.norm(linear_velocity_r[:2]) * 10

        return vlinear_norm_l, vlinear_norm_r
    except Exception as e:
        raise e
    
def difference_radians(rad1, rad2):
    """ difference between two radians """
    
    try:
        if rad2 < 0:
            rad2 = 2*np.pi + rad2
            
        if rad1 < 0:
            rad1 = 2*np.pi + rad1
        
        if 0 <= rad2 <= pi and pi <= rad1 <= 2*pi or rad2 > rad1:
            rad = pi - abs(abs(rad2 - rad1) - pi)
        else:
            rad = -(pi - abs(abs(rad2 - rad1) - pi))
        return rad
    
    except Exception as e:
        raise e
        
def read_sonar_sensors(ini=1, end=N_SENSORS+1):
    # reads range of sonar sensors
    
    try:
        DetectedPoints = []
        Distances = []
        for i in range(ini,end):
            res, status, detectedpoint,_,_ = vrep.simxReadProximitySensor(clientID, sonar_sensor_handles[i-1], vrep.simx_opmode_streaming)
            while(res != vrep.simx_return_ok):
                res, status, detectedpoint,_,_ = vrep.simxReadProximitySensor(clientID, sonar_sensor_handles[i-1], vrep.simx_opmode_buffer)

            if(status != 0):
                DetectedPoints.append(detectedpoint)
                Distances.append( round(detectedpoint[2],5) )
            else:
                DetectedPoints.append([100,100,100])
                Distances.append(NO_DETECTION_DIST)

        return DetectedPoints, Distances
    except Exception as e:
        raise e

def get_position(object_handle, parent_handle):
    """ get the positions of an object in relation to a parent """
    
    try:
        res, position = vrep.simxGetObjectPosition(clientID, object_handle, parent_handle, vrep.simx_opmode_streaming)
        while(res != vrep.simx_return_ok):
            res, position = vrep.simxGetObjectPosition(clientID, object_handle, parent_handle, vrep.simx_opmode_streaming)
    
        return position
    except Exception as e:
        raise e

def get_orientation(object_handle, parent_handle):
    """ get the orientation of an object in relation to a parent """
    
    try:
        res, ori = vrep.simxGetObjectOrientation(clientID, object_handle, parent_handle, vrep.simx_opmode_streaming)
        while(res != vrep.simx_return_ok):
            res, ori = vrep.simxGetObjectOrientation(clientID, object_handle, parent_handle, vrep.simx_opmode_buffer)

        return ori
    except Exception as e:
        raise e

def get_joint_position(object_handle):
    
    try:
        err_code, angle = vrep.simxGetJointPosition(clientID, object_handle, vrep.simx_opmode_streaming)
        while err_code != vrep.simx_return_ok:
            err_code, angle = vrep.simxGetJointPosition(clientID, object_handle, vrep.simx_opmode_buffer)

        return angle
    except Exception as e:
        raise e
    
def odometry_new_pos(ini_pos, left_rad, right_rad):
    try:
        delta_s = WHEEL_RADIUS * (left_rad + right_rad) / 2
        delta_theta = WHEEL_RADIUS * (right_rad - left_rad) / DISTANCE_BETWEEN_WHEELS

        new_pos = [0.0,0.0,0.0]
        new_pos[0] = ini_pos[0] + (delta_s * cos(ini_pos[2] + (delta_theta/2)))
        new_pos[1] = ini_pos[1] + (delta_s * sin(ini_pos[2] + (delta_theta/2)))
        new_pos[2] = ini_pos[2] + delta_theta

        return new_pos
    except Exception as e:
        raise e
        
def collect_odometry(ini_pos, ini_left_angle, ini_right_angle):
    try:
        new_left_angle = get_joint_position(l_motor_handle)
        new_right_angle = get_joint_position(r_motor_handle)

        left_angle_diff = difference_radians(ini_left_angle, new_left_angle)
        right_angle_diff = difference_radians(ini_right_angle, new_right_angle)

        robot_new_odometry = odometry_new_pos(ini_pos, left_angle_diff, right_angle_diff)
        #print(robot_new_odometry[:3], robot_pos_odometry[:3], robot_pos_gt[:3], ini_left_angle,
              #new_left_angle, left_angle_diff, ini_right_angle, new_right_angle, right_angle_diff)

        position_odometry.append(robot_new_odometry)

        return robot_new_odometry, new_left_angle, new_right_angle
    except Exception as e:
        raise e

def collect_ground_truth():
    try:
        robot_pos_gt = get_position(robot_handle,-1)[:2] + [get_orientation(robot_handle, -1)[2]]
        position_ground_truth.append(robot_pos_gt)
        return robot_pos_gt
    except Exception as e:
        raise e
        
        
def points_transformation_collection(points,trans_matrix, points_source):
    """
        Transform points collected by sensors to global coordinate and recollect all the points
    """

    TR = np.array([[np.cos(trans_matrix[2]), -np.sin(trans_matrix[2]), trans_matrix[0]],
                   [np.sin(trans_matrix[2]),  np.cos(trans_matrix[2]), trans_matrix[1]],
                   [0,                        0,                       1             ]])

    for p in range(len(points)):

        if points[p][0] == 100:
            continue

        # print(len(points), TS[p], p)
        
        collected_points_global = np.dot(TR,TS[p].dot(np.asarray(points[p])))
        

        if points_source == 'gt':
            points_cloud_ground_truth.append(collected_points_global)  
        elif points_source == 'od':
            points_cloud_odometry.append(collected_points_global)
        

def walk_straigh_meters(m, end_vel=0, robot_pos_odometry=[0,0,0]):
    """ the robot walks a pre-determined quantity of meters """
    
    try:
        wheels_radians_required = m / (WHEEL_DIAMETER / 2)
        set_velocity_both_motors(ROBOT_VELOCITY)
        
        ini_left_angle = get_joint_position(l_motor_handle)
        ini_right_angle = get_joint_position(r_motor_handle)
        
        while wheels_radians_required > 0:
            # robot position - ground truth (x, y, theta)
            robot_pos_gt = collect_ground_truth()

            # robot odometry
            robot_new_odometry, new_left_angle, new_right_angle = collect_odometry(robot_pos_odometry, 
                                                                                   ini_left_angle,
                                                                                   ini_right_angle)
                    
            robot_pos_odometry = robot_new_odometry
            
            wheels_radians_required -= difference_radians(ini_left_angle, new_left_angle)
            
            ini_left_angle = new_left_angle
            ini_right_angle = new_right_angle
            
            all_detected_points, all_distances = read_sonar_sensors()
            pos = get_position(robot_handle,-1)
            salva_dados(all_distances, pos[0], pos[1], robot_new_odometry[0], robot_new_odometry[1], get_orientation(robot_handle,-1)[2])
            
            points_transformation_collection(all_detected_points, position_ground_truth[-1], 'gt')
            points_transformation_collection(all_detected_points, position_odometry[-1], 'od')
            
            if all_distances[3] < NO_DETECTION_DIST / 2:
                print('Object detected. Stopping...')
                wheels_radians_required = 0
                set_velocity_both_motors(0)
            
            time.sleep(INTERVAL_CHECK_POSITION)
            
        set_velocity_both_motors(0)
        
        return robot_pos_odometry
    except Exception as e:
        raise e
        

def spin_robot(angle, direction='l', odo_current_position=[0,0,0], v=0):
    """ turns the robot to an specific angle 
        (is it yaw movement?)
        angle in radians
    """
    try:
        direction = direction.lower()

        if direction in ("left", "l"):
            m_0, m_1 = "l", "r"
            motor_handle = r_motor_handle
        elif direction in ("right", "r"):
            m_0, m_1 = "r", "l"
            motor_handle = l_motor_handle

        # initial radian position of the wheel
        ini_angle = get_joint_position(motor_handle)

        wheels_radians_required = (DISTANCE_BETWEEN_WHEELS / 2) * angle / (WHEEL_DIAMETER / 2)

        set_velocity_one_motor(m_0, -ROBOT_VELOCITY)
        set_velocity_one_motor(m_1, ROBOT_VELOCITY)

        while wheels_radians_required > 0:
            new_angle = get_joint_position(motor_handle)

            diff_radians = difference_radians(ini_angle, new_angle)
            wheels_radians_required -= diff_radians

            collect_ground_truth()

            if m_1 == 'l':
                odo_new_position = odometry_new_pos(odo_current_position, diff_radians, -diff_radians)
            elif m_1 == 'r':
                odo_new_position = odometry_new_pos(odo_current_position, -diff_radians, diff_radians)

            odo_current_position = odo_new_position
            position_odometry.append(odo_current_position)

            ini_angle = new_angle

            time.sleep(INTERVAL_CHECK_POSITION)

        if v is None:
            set_velocity_one_motor(m_0, vlinear_0)
            set_velocity_one_motor(m_1, vlinear_1)
        else:
            set_velocity_both_motors(v)

        return odo_current_position
    
    except Exception as e:
        raise e

In [ ]:
braitenbergL=[-0.2,-0.4,-0.6,-0.8,-1.0,-1.2,-1.4,-1.6]
braitenbergR=[-1.6,-1.4,-1.2,-1.0,-0.8,-0.6,-0.4,-0.2]

detect = [0,0,0,0,0,0,0,0]
noDetectionDist = 1.0
maxDetectionDist = 0.2

R = 0.0975
L = 0.36205

def braitenberg(dist, vel):
    """
        Control the robot movement by the distances read with the ultrassonic sensors. More info: https://en.wikipedia.org/wiki/Braitenberg_vehicle
        Args:
            dist: Ultrassonic distances list
            vel:  Max wheel velocities
    """
    
    try:
        vLeft = vRight = vel
        for i in range(len(dist)):
            if(dist[i] < noDetectionDist):
                detect[i] = 1 - ((dist[i]-maxDetectionDist)/(noDetectionDist-maxDetectionDist))
            else:
                detect[i]=0
            for i in range(8):
                vLeft = vLeft + braitenbergL[i]*detect[i]
                vRight = vRight+ braitenbergR[i]*detect[i]

        return [vLeft, vRight]
    except Exception as e:
        raise e
        
def path_braitenberg():
    """ main function concerning the path that the robot is going to make """
    
    # Inner function that will be set to be executed when the robot should stop
    def handler_timeout(signum, frame):
        """ just a timeout handler :) """
        raise Exception

    try:
        
        # Set the signal to stop the robot in ROBOT_TIMEOUT seconds
        signal.signal(signal.SIGALRM, handler_timeout)
        signal.alarm(ROBOT_TIMEOUT)

        # call record_robot_ground_truth each INTERVAL_CHECK_POSITION seconds
        robot_pos_odometry = ROBOT_INIT_XY + [ROBOT_INIT_THETA]

        ini_left_angle = get_joint_position(l_motor_handle)
        ini_right_angle = get_joint_position(r_motor_handle)
        
        start_time = time.time()
        while True:
            # robot position - ground truth (x, y, theta)
            robot_pos_gt = collect_ground_truth()

            # robot odometry
            robot_new_odometry, new_left_angle, new_right_angle = collect_odometry(robot_pos_odometry, 
                                                                                   ini_left_angle,
                                                                                   ini_right_angle)
                    
            robot_pos_odometry = robot_new_odometry
            
            ini_left_angle = new_left_angle
            ini_right_angle = new_right_angle

            # Reads the sonar sensors
            
            all_detected_points, all_distances = read_sonar_sensors()
            points_transformation_collection(all_detected_points, position_ground_truth[-1], 'gt')
            points_transformation_collection(all_detected_points, position_odometry[-1], 'od')
            
            # Calculate the new velocities of each motor
            vel = braitenberg(all_distances[:8], ROBOT_VELOCITY) #Using only the 8 frontal sensors
        
            # If at least one velocity is negative, turns the robot 15º until both
            # velocities are positive
            while vel[0] < 0 or vel[1] < 0:
                robot_new_odometry = spin_robot(np.pi/12, 'l', robot_pos_odometry, ROBOT_VELOCITY)
                
                robot_pos_odometry = robot_new_odometry
                ini_left_angle = get_joint_position(l_motor_handle)
                ini_right_angle = get_joint_position(r_motor_handle)
                
                all_detected_points, all_distances = read_sonar_sensors()
                points_transformation_collection(all_detected_points, position_ground_truth[-1], 'gt')
                points_transformation_collection(all_detected_points, position_odometry[-1], 'od')
                
                vel = braitenberg(all_distances[:8], ROBOT_VELOCITY)
            
            set_velocity_one_motor('l',vel[0])
            set_velocity_one_motor('r',vel[1])
            
            time.sleep(INTERVAL_CHECK_POSITION)
            
    except Exception as e:
        print(e)
        set_velocity_both_motors(0)

def planned_path():
    """ main function concerning the path that the robot is going to make """

    try:

        # call record_robot_ground_truth each INTERVAL_CHECK_POSITION seconds
        robot_pos_odometry = ROBOT_INIT_XY + [ROBOT_INIT_THETA]
        position_odometry.append(robot_pos_odometry)
        i = 0
        
        while True:
            dist = r.randint(1, 6)
            print('Walking {}m.'.format(dist))
            robot_pos_odometry = walk_straigh_meters(dist, 0, robot_pos_odometry)
            
            direct = 'l' if r.randint(0, 1) else 'r'
            ang = np.pi/r.randint(2, 12)
            print('Spining {} deg on {}'.format(int(ang*180.0/np.pi), direct))
            robot_pos_odometry = spin_robot(ang, direct, robot_pos_odometry)
            
            if i > 0 and i % 2 == 0:
                plotar_mapa()
            i += 1
             
    except Exception as e:
        raise e
        set_velocity_both_motors(0)

In [ ]:
planned_path()
# path_braitenberg()

In [ ]:
plotar_mapa()

In [ ]:
def plot_points_map(points, title, lims=True):
    plt.figure(dpi=1080)
    plt.plot(*zip(*points))
    
    if lims == True:
        plt.xlim(X_LEFT_LIM, X_RIGHT_LIM)
        plt.ylim(Y_BOTTOM_LIM, Y_UPPER_LIM)
    plt.title(title)
    plt.show()

def plot_points_map_2(points1, points2, title, lims=True):
    plt.figure(dpi=1080)
    plt.scatter(*zip(*points1),c='#bcbd22',s=1)
    plt.scatter(*zip(*points2),c='#17becf',s=1)
    
    if lims == True:
        plt.xlim(X_LEFT_LIM, X_RIGHT_LIM)
        plt.ylim(Y_BOTTOM_LIM, Y_UPPER_LIM)
    plt.title(title)
    plt.show()


# plot_points_map(position_ground_truth, "gt")
# plot_points_map(position_odometry, "od")
    
plot_points_map_2(np.asarray(position_ground_truth)[:,:2], np.asarray(position_odometry)[:,:2], "Path")

def plot_points_map_scatter(path, points, title, lims=True):
    plt.figure(dpi=1080)
#     plt.plot(*zip(*path))
    plt.scatter(*zip(*points),c='#bcbd22',s=1)
    if lims == True:
        plt.xlim(X_LEFT_LIM, X_RIGHT_LIM)
        plt.ylim(Y_BOTTOM_LIM, Y_UPPER_LIM)
#     plt.title(title)
#     plt.show()
#    plt.axis('off')
#    plt.savefig('fig_sensor.tiff',bbox_inches='tight')

plot_points_map_scatter(np.asarray(position_ground_truth)[:,:2], 
                     np.asarray(points_cloud_ground_truth)[:,:2], 
                     "Ground Truth",lims=True)

plot_points_map_scatter(np.asarray(position_odometry)[:,:2], 
                     np.asarray(points_cloud_odometry)[:,:2], 
                     "Odometry",lims=True)

In [ ]:
position_odometry

In [ ]:
position_ground_truth

In [ ]:
def plot():
    plt.scatter(10, 10, s=0.5)
    odom, = plt.plot(*zip(*position_odometry), 'r--', label='Odometry')
    path, = plt.plot(*zip(*position_ground_truth), 'g', label='Ground-truth')
    plt.legend(handles=[odom, path])
    plt.show()

In [ ]:
plot()
